In [30]:
import json
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def getAllCurrentEntityRevisions():
    results = query("""
        select
            max(entity_revision.id),
            max(entity.id),
            max(type.name),
            json_objectagg(entity_revision_field.field, entity_revision_field.value) as fields
        from entity
            join type on entity.type_id = type.id
            join entity_revision on entity_revision.id = entity.current_revision_id
            join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        group by entity_revision.id
    """)
    
    return [{
        "currentRevisionId": result[0],
        "entityId": result[1],
        "type": result[2],
        **json.loads(result[3])
    } for result in results]

getAllCurrentEntityRevisions()[0]

{'currentRevisionId': 1568,
 'entityId': 1567,
 'type': 'article',
 'title': 'Geschwindigkeit',
 'content': '[[{"col":24,"content":"Die Geschwindigkeit ist ein Ma\\u00df daf\\u00fcr, wie schnell sich ein K\\u00f6rper bewegt.\\n\\n\\u00a0\\n\\n\\n# Bekannte Ma\\u00dfeinheiten\\n\\n"}],[{"col":8,"content":"%%\\\\frac ms%%"},{"col":8,"content":"\\n\\n [Meter](/1681)  pro Sekunde"},{"col":8,"content":"\\n\\n%%1\\\\frac ms=1\\\\cdot3,6\\\\frac{\\\\mathrm{km}}h=3,6\\\\;\\\\frac{\\\\mathrm{km}}h%%"}],[{"col":8,"content":"\\n\\n%%\\\\frac{\\\\mathrm{km}}h%%"},{"col":8,"content":"\\n\\n [Kilometer](/1681)  pro Stunde"},{"col":8,"content":"\\n\\n\\u00a0"}],[{"col":8,"content":"\\n\\nmph"},{"col":8,"content":"\\n\\n [Meilen](/1681)  pro Stunde"},{"col":8,"content":"\\n\\n%%1\\\\frac{\\\\mathrm{km}}h=0,62\\\\;\\\\mathrm{mph}%%"}],[{"col":8,"content":"\\n\\nkn \\/ %%\\\\frac{\\\\mathrm{sm}}h%%"},{"col":8,"content":"\\n\\nKnoten \\/  [Seemeilen](/1681)  pro Stunde"},{"col":8,"content":"\\n\\n%%1\\\\

In [31]:
def parseEdtrState(content):
    try:
        state = json.loads(content)
    except:
        return None
    
    if "plugin" not in state:
        return None
    
    return state

def stateHasPlugin(plugin, state):
    if isinstance(state, dict):
        if "plugin" in state and state["plugin"] == plugin:
            return True
        
        return any(stateHasPlugin(plugin, value) for value in state.values())
    elif isinstance(state, list):
        return any(stateHasPlugin(plugin, value) for value in state)
    else:
        return False

def contentHasPlugin(plugin, content):
    state = parseEdtrState(content)
    
    return stateHasPlugin(plugin, state) if state != None else False

content = getAllCurrentEntityRevisions()[0]["content"]

print(contentHasPlugin("text", content))
print(contentHasPlugin("important", content))

False
False


In [34]:
def printRevisions(revisions):
    def toMarkdown(revision):
        return "* [%s %s](https://serlo-staging.dev/%s) ([current revision %s](https://serlo-staging.dev/%s))" % (
            revision["type"], revision["entityId"], revision["entityId"],
            revision["currentRevisionId"], revision["currentRevisionId"]
        )
    
    code = "\n".join((toMarkdown(r) for r in revisions))
    display(Markdown(code))

def printRevisionsWithPlugin(plugin):
    display(Markdown(f"# Entites with plugin {plugin}"))
    
    revisions = getAllCurrentEntityRevisions()
    
    printRevisions((r for r in revisions if "content" in r and contentHasPlugin(plugin, r["content"])))
    
printRevisionsWithPlugin("important")
printRevisionsWithPlugin("blockquote")

# Entites with plugin important

* [course-page 143526](https://serlo-staging.dev/143526) ([current revision 144759](https://serlo-staging.dev/144759))
* [article 146841](https://serlo-staging.dev/146841) ([current revision 147378](https://serlo-staging.dev/147378))
* [event 145455](https://serlo-staging.dev/145455) ([current revision 147732](https://serlo-staging.dev/147732))
* [event 146675](https://serlo-staging.dev/146675) ([current revision 147742](https://serlo-staging.dev/147742))
* [text-exercise 60274](https://serlo-staging.dev/60274) ([current revision 147998](https://serlo-staging.dev/147998))
* [article 1781](https://serlo-staging.dev/1781) ([current revision 148027](https://serlo-staging.dev/148027))
* [article 148316](https://serlo-staging.dev/148316) ([current revision 148318](https://serlo-staging.dev/148318))
* [course-page 61686](https://serlo-staging.dev/61686) ([current revision 149204](https://serlo-staging.dev/149204))
* [course-page 61810](https://serlo-staging.dev/61810) ([current revision 149206](https://serlo-staging.dev/149206))
* [event 147532](https://serlo-staging.dev/147532) ([current revision 149743](https://serlo-staging.dev/149743))
* [event 147705](https://serlo-staging.dev/147705) ([current revision 149744](https://serlo-staging.dev/149744))
* [article 65164](https://serlo-staging.dev/65164) ([current revision 151250](https://serlo-staging.dev/151250))
* [course-page 128986](https://serlo-staging.dev/128986) ([current revision 152468](https://serlo-staging.dev/152468))
* [text-exercise 6705](https://serlo-staging.dev/6705) ([current revision 152977](https://serlo-staging.dev/152977))
* [event 149177](https://serlo-staging.dev/149177) ([current revision 153541](https://serlo-staging.dev/153541))
* [article 1949](https://serlo-staging.dev/1949) ([current revision 153839](https://serlo-staging.dev/153839))
* [article 154812](https://serlo-staging.dev/154812) ([current revision 154949](https://serlo-staging.dev/154949))
* [article 154665](https://serlo-staging.dev/154665) ([current revision 154991](https://serlo-staging.dev/154991))
* [course-page 127669](https://serlo-staging.dev/127669) ([current revision 155000](https://serlo-staging.dev/155000))
* [course-page 147269](https://serlo-staging.dev/147269) ([current revision 155019](https://serlo-staging.dev/155019))
* [course-page 73894](https://serlo-staging.dev/73894) ([current revision 156214](https://serlo-staging.dev/156214))
* [course-page 72987](https://serlo-staging.dev/72987) ([current revision 156302](https://serlo-staging.dev/156302))
* [course-page 156432](https://serlo-staging.dev/156432) ([current revision 156506](https://serlo-staging.dev/156506))
* [course-page 156410](https://serlo-staging.dev/156410) ([current revision 156519](https://serlo-staging.dev/156519))
* [text-exercise 60191](https://serlo-staging.dev/60191) ([current revision 156775](https://serlo-staging.dev/156775))
* [text-exercise 60216](https://serlo-staging.dev/60216) ([current revision 156782](https://serlo-staging.dev/156782))
* [course-page 156517](https://serlo-staging.dev/156517) ([current revision 156813](https://serlo-staging.dev/156813))
* [course-page 156507](https://serlo-staging.dev/156507) ([current revision 156855](https://serlo-staging.dev/156855))
* [course-page 156511](https://serlo-staging.dev/156511) ([current revision 156864](https://serlo-staging.dev/156864))
* [course-page 127666](https://serlo-staging.dev/127666) ([current revision 157162](https://serlo-staging.dev/157162))
* [article 152323](https://serlo-staging.dev/152323) ([current revision 157605](https://serlo-staging.dev/157605))
* [text-solution 101450](https://serlo-staging.dev/101450) ([current revision 158179](https://serlo-staging.dev/158179))
* [text-solution 70167](https://serlo-staging.dev/70167) ([current revision 158228](https://serlo-staging.dev/158228))
* [course-page 155806](https://serlo-staging.dev/155806) ([current revision 158376](https://serlo-staging.dev/158376))
* [text-solution 25961](https://serlo-staging.dev/25961) ([current revision 158442](https://serlo-staging.dev/158442))
* [article 154244](https://serlo-staging.dev/154244) ([current revision 159223](https://serlo-staging.dev/159223))
* [article 158346](https://serlo-staging.dev/158346) ([current revision 159271](https://serlo-staging.dev/159271))
* [article 152450](https://serlo-staging.dev/152450) ([current revision 159451](https://serlo-staging.dev/159451))
* [article 158031](https://serlo-staging.dev/158031) ([current revision 160200](https://serlo-staging.dev/160200))
* [text-solution 3083](https://serlo-staging.dev/3083) ([current revision 160430](https://serlo-staging.dev/160430))
* [text-solution 3263](https://serlo-staging.dev/3263) ([current revision 160780](https://serlo-staging.dev/160780))
* [article 153419](https://serlo-staging.dev/153419) ([current revision 160793](https://serlo-staging.dev/160793))
* [article 127338](https://serlo-staging.dev/127338) ([current revision 160838](https://serlo-staging.dev/160838))
* [article 154653](https://serlo-staging.dev/154653) ([current revision 160851](https://serlo-staging.dev/160851))
* [course-page 161361](https://serlo-staging.dev/161361) ([current revision 161373](https://serlo-staging.dev/161373))
* [text-exercise-group 5259](https://serlo-staging.dev/5259) ([current revision 161446](https://serlo-staging.dev/161446))
* [course-page 162051](https://serlo-staging.dev/162051) ([current revision 162071](https://serlo-staging.dev/162071))
* [course-page 162048](https://serlo-staging.dev/162048) ([current revision 162176](https://serlo-staging.dev/162176))
* [article 25812](https://serlo-staging.dev/25812) ([current revision 163795](https://serlo-staging.dev/163795))
* [article 54534](https://serlo-staging.dev/54534) ([current revision 164224](https://serlo-staging.dev/164224))
* [article 145927](https://serlo-staging.dev/145927) ([current revision 164607](https://serlo-staging.dev/164607))
* [article 165223](https://serlo-staging.dev/165223) ([current revision 165746](https://serlo-staging.dev/165746))
* [course-page 139538](https://serlo-staging.dev/139538) ([current revision 165776](https://serlo-staging.dev/165776))
* [article 163681](https://serlo-staging.dev/163681) ([current revision 166131](https://serlo-staging.dev/166131))
* [course-page 44099](https://serlo-staging.dev/44099) ([current revision 167115](https://serlo-staging.dev/167115))
* [article 167149](https://serlo-staging.dev/167149) ([current revision 167627](https://serlo-staging.dev/167627))
* [article 167635](https://serlo-staging.dev/167635) ([current revision 167790](https://serlo-staging.dev/167790))
* [course-page 51853](https://serlo-staging.dev/51853) ([current revision 168104](https://serlo-staging.dev/168104))
* [course-page 167929](https://serlo-staging.dev/167929) ([current revision 168111](https://serlo-staging.dev/168111))
* [course-page 168455](https://serlo-staging.dev/168455) ([current revision 168532](https://serlo-staging.dev/168532))
* [article 168274](https://serlo-staging.dev/168274) ([current revision 168855](https://serlo-staging.dev/168855))
* [article 168325](https://serlo-staging.dev/168325) ([current revision 169110](https://serlo-staging.dev/169110))
* [event 145464](https://serlo-staging.dev/145464) ([current revision 169335](https://serlo-staging.dev/169335))
* [event 145458](https://serlo-staging.dev/145458) ([current revision 169336](https://serlo-staging.dev/169336))
* [event 145459](https://serlo-staging.dev/145459) ([current revision 169337](https://serlo-staging.dev/169337))
* [event 147733](https://serlo-staging.dev/147733) ([current revision 169339](https://serlo-staging.dev/169339))
* [text-solution 94860](https://serlo-staging.dev/94860) ([current revision 169393](https://serlo-staging.dev/169393))
* [text-solution 158666](https://serlo-staging.dev/158666) ([current revision 169628](https://serlo-staging.dev/169628))
* [article 168166](https://serlo-staging.dev/168166) ([current revision 169647](https://serlo-staging.dev/169647))
* [course-page 169721](https://serlo-staging.dev/169721) ([current revision 169734](https://serlo-staging.dev/169734))
* [course-page 169722](https://serlo-staging.dev/169722) ([current revision 169735](https://serlo-staging.dev/169735))
* [course-page 169724](https://serlo-staging.dev/169724) ([current revision 169737](https://serlo-staging.dev/169737))
* [course-page 169725](https://serlo-staging.dev/169725) ([current revision 169738](https://serlo-staging.dev/169738))
* [course-page 169727](https://serlo-staging.dev/169727) ([current revision 169814](https://serlo-staging.dev/169814))
* [article 168248](https://serlo-staging.dev/168248) ([current revision 170414](https://serlo-staging.dev/170414))
* [article 168282](https://serlo-staging.dev/168282) ([current revision 170417](https://serlo-staging.dev/170417))
* [article 167755](https://serlo-staging.dev/167755) ([current revision 170467](https://serlo-staging.dev/170467))
* [article 167769](https://serlo-staging.dev/167769) ([current revision 170533](https://serlo-staging.dev/170533))
* [article 168363](https://serlo-staging.dev/168363) ([current revision 170623](https://serlo-staging.dev/170623))
* [article 168400](https://serlo-staging.dev/168400) ([current revision 170632](https://serlo-staging.dev/170632))
* [article 170734](https://serlo-staging.dev/170734) ([current revision 170805](https://serlo-staging.dev/170805))
* [article 167910](https://serlo-staging.dev/167910) ([current revision 170815](https://serlo-staging.dev/170815))
* [article 166154](https://serlo-staging.dev/166154) ([current revision 171150](https://serlo-staging.dev/171150))
* [article 25507](https://serlo-staging.dev/25507) ([current revision 171440](https://serlo-staging.dev/171440))
* [article 54317](https://serlo-staging.dev/54317) ([current revision 171699](https://serlo-staging.dev/171699))
* [text-solution 86758](https://serlo-staging.dev/86758) ([current revision 172534](https://serlo-staging.dev/172534))
* [text-solution 86761](https://serlo-staging.dev/86761) ([current revision 172536](https://serlo-staging.dev/172536))
* [text-solution 86763](https://serlo-staging.dev/86763) ([current revision 172538](https://serlo-staging.dev/172538))
* [article 169299](https://serlo-staging.dev/169299) ([current revision 173411](https://serlo-staging.dev/173411))
* [article 165740](https://serlo-staging.dev/165740) ([current revision 173425](https://serlo-staging.dev/173425))
* [article 168538](https://serlo-staging.dev/168538) ([current revision 173433](https://serlo-staging.dev/173433))
* [grouped-text-exercise 173786](https://serlo-staging.dev/173786) ([current revision 173796](https://serlo-staging.dev/173796))
* [article 32200](https://serlo-staging.dev/32200) ([current revision 173878](https://serlo-staging.dev/173878))
* [article 168256](https://serlo-staging.dev/168256) ([current revision 175193](https://serlo-staging.dev/175193))
* [text-solution 52244](https://serlo-staging.dev/52244) ([current revision 176295](https://serlo-staging.dev/176295))
* [course-page 176725](https://serlo-staging.dev/176725) ([current revision 176735](https://serlo-staging.dev/176735))
* [course-page 176727](https://serlo-staging.dev/176727) ([current revision 176737](https://serlo-staging.dev/176737))
* [course-page 176729](https://serlo-staging.dev/176729) ([current revision 176739](https://serlo-staging.dev/176739))
* [course-page 177860](https://serlo-staging.dev/177860) ([current revision 177879](https://serlo-staging.dev/177879))
* [course-page 177858](https://serlo-staging.dev/177858) ([current revision 177881](https://serlo-staging.dev/177881))
* [article 178239](https://serlo-staging.dev/178239) ([current revision 178299](https://serlo-staging.dev/178299))
* [article 178269](https://serlo-staging.dev/178269) ([current revision 178722](https://serlo-staging.dev/178722))
* [article 178847](https://serlo-staging.dev/178847) ([current revision 178903](https://serlo-staging.dev/178903))
* [course-page 179170](https://serlo-staging.dev/179170) ([current revision 179196](https://serlo-staging.dev/179196))
* [course-page 178850](https://serlo-staging.dev/178850) ([current revision 179246](https://serlo-staging.dev/179246))
* [course-page 178866](https://serlo-staging.dev/178866) ([current revision 179251](https://serlo-staging.dev/179251))
* [event 179657](https://serlo-staging.dev/179657) ([current revision 179698](https://serlo-staging.dev/179698))
* [course-page 179753](https://serlo-staging.dev/179753) ([current revision 179756](https://serlo-staging.dev/179756))
* [course-page 179759](https://serlo-staging.dev/179759) ([current revision 179802](https://serlo-staging.dev/179802))
* [article 167497](https://serlo-staging.dev/167497) ([current revision 179907](https://serlo-staging.dev/179907))
* [article 178254](https://serlo-staging.dev/178254) ([current revision 180002](https://serlo-staging.dev/180002))
* [event 145590](https://serlo-staging.dev/145590) ([current revision 180193](https://serlo-staging.dev/180193))
* [article 178258](https://serlo-staging.dev/178258) ([current revision 180707](https://serlo-staging.dev/180707))
* [article 178261](https://serlo-staging.dev/178261) ([current revision 180773](https://serlo-staging.dev/180773))
* [article 179744](https://serlo-staging.dev/179744) ([current revision 180839](https://serlo-staging.dev/180839))
* [article 179771](https://serlo-staging.dev/179771) ([current revision 180845](https://serlo-staging.dev/180845))
* [article 179780](https://serlo-staging.dev/179780) ([current revision 180850](https://serlo-staging.dev/180850))
* [course-page 179762](https://serlo-staging.dev/179762) ([current revision 180852](https://serlo-staging.dev/180852))
* [article 179789](https://serlo-staging.dev/179789) ([current revision 180853](https://serlo-staging.dev/180853))
* [course-page 19495](https://serlo-staging.dev/19495) ([current revision 181887](https://serlo-staging.dev/181887))
* [article 27823](https://serlo-staging.dev/27823) ([current revision 181889](https://serlo-staging.dev/181889))
* [article 1855](https://serlo-staging.dev/1855) ([current revision 183896](https://serlo-staging.dev/183896))
* [event 169327](https://serlo-staging.dev/169327) ([current revision 183938](https://serlo-staging.dev/183938))
* [course-page 167674](https://serlo-staging.dev/167674) ([current revision 184084](https://serlo-staging.dev/184084))
* [course-page 168343](https://serlo-staging.dev/168343) ([current revision 184086](https://serlo-staging.dev/184086))
* [course-page 168491](https://serlo-staging.dev/168491) ([current revision 184090](https://serlo-staging.dev/184090))
* [article 170363](https://serlo-staging.dev/170363) ([current revision 184688](https://serlo-staging.dev/184688))
* [article 98333](https://serlo-staging.dev/98333) ([current revision 184859](https://serlo-staging.dev/184859))
* [article 184028](https://serlo-staging.dev/184028) ([current revision 185563](https://serlo-staging.dev/185563))
* [article 181279](https://serlo-staging.dev/181279) ([current revision 185617](https://serlo-staging.dev/185617))
* [course-page 186587](https://serlo-staging.dev/186587) ([current revision 186596](https://serlo-staging.dev/186596))
* [course-page 186589](https://serlo-staging.dev/186589) ([current revision 186598](https://serlo-staging.dev/186598))
* [course-page 186591](https://serlo-staging.dev/186591) ([current revision 186600](https://serlo-staging.dev/186600))
* [article 186610](https://serlo-staging.dev/186610) ([current revision 186623](https://serlo-staging.dev/186623))
* [article 90785](https://serlo-staging.dev/90785) ([current revision 186661](https://serlo-staging.dev/186661))
* [article 107863](https://serlo-staging.dev/107863) ([current revision 186662](https://serlo-staging.dev/186662))
* [course-page 187203](https://serlo-staging.dev/187203) ([current revision 187216](https://serlo-staging.dev/187216))
* [course-page 188326](https://serlo-staging.dev/188326) ([current revision 188385](https://serlo-staging.dev/188385))
* [article 187487](https://serlo-staging.dev/187487) ([current revision 190508](https://serlo-staging.dev/190508))
* [article 181056](https://serlo-staging.dev/181056) ([current revision 191176](https://serlo-staging.dev/191176))
* [text-solution 190460](https://serlo-staging.dev/190460) ([current revision 191195](https://serlo-staging.dev/191195))
* [text-solution 190462](https://serlo-staging.dev/190462) ([current revision 191197](https://serlo-staging.dev/191197))
* [article 155311](https://serlo-staging.dev/155311) ([current revision 191244](https://serlo-staging.dev/191244))
* [article 185678](https://serlo-staging.dev/185678) ([current revision 191534](https://serlo-staging.dev/191534))
* [course-page 156422](https://serlo-staging.dev/156422) ([current revision 191596](https://serlo-staging.dev/191596))
* [text-solution 190464](https://serlo-staging.dev/190464) ([current revision 191849](https://serlo-staging.dev/191849))
* [article 1859](https://serlo-staging.dev/1859) ([current revision 192445](https://serlo-staging.dev/192445))
* [article 1803](https://serlo-staging.dev/1803) ([current revision 192807](https://serlo-staging.dev/192807))
* [article 1937](https://serlo-staging.dev/1937) ([current revision 192821](https://serlo-staging.dev/192821))
* [course-page 36766](https://serlo-staging.dev/36766) ([current revision 193043](https://serlo-staging.dev/193043))
* [course-page 36833](https://serlo-staging.dev/36833) ([current revision 193050](https://serlo-staging.dev/193050))
* [course-page 36825](https://serlo-staging.dev/36825) ([current revision 193053](https://serlo-staging.dev/193053))
* [article 62547](https://serlo-staging.dev/62547) ([current revision 193082](https://serlo-staging.dev/193082))
* [course-page 62226](https://serlo-staging.dev/62226) ([current revision 193084](https://serlo-staging.dev/193084))
* [course-page 99544](https://serlo-staging.dev/99544) ([current revision 193280](https://serlo-staging.dev/193280))
* [article 90405](https://serlo-staging.dev/90405) ([current revision 193490](https://serlo-staging.dev/193490))
* [article 51369](https://serlo-staging.dev/51369) ([current revision 193749](https://serlo-staging.dev/193749))
* [article 174232](https://serlo-staging.dev/174232) ([current revision 193814](https://serlo-staging.dev/193814))
* [course-page 193816](https://serlo-staging.dev/193816) ([current revision 193841](https://serlo-staging.dev/193841))
* [course-page 193817](https://serlo-staging.dev/193817) ([current revision 193842](https://serlo-staging.dev/193842))
* [article 181058](https://serlo-staging.dev/181058) ([current revision 194425](https://serlo-staging.dev/194425))
* [article 77792](https://serlo-staging.dev/77792) ([current revision 194907](https://serlo-staging.dev/194907))
* [article 194758](https://serlo-staging.dev/194758) ([current revision 194970](https://serlo-staging.dev/194970))
* [article 1625](https://serlo-staging.dev/1625) ([current revision 194972](https://serlo-staging.dev/194972))
* [article 195043](https://serlo-staging.dev/195043) ([current revision 195045](https://serlo-staging.dev/195045))
* [course-page 178835](https://serlo-staging.dev/178835) ([current revision 195226](https://serlo-staging.dev/195226))
* [course-page 195294](https://serlo-staging.dev/195294) ([current revision 195308](https://serlo-staging.dev/195308))
* [course-page 195306](https://serlo-staging.dev/195306) ([current revision 195311](https://serlo-staging.dev/195311))
* [course-page 195318](https://serlo-staging.dev/195318) ([current revision 195320](https://serlo-staging.dev/195320))
* [article 1579](https://serlo-staging.dev/1579) ([current revision 195354](https://serlo-staging.dev/195354))
* [article 195041](https://serlo-staging.dev/195041) ([current revision 195469](https://serlo-staging.dev/195469))
* [course-page 196540](https://serlo-staging.dev/196540) ([current revision 196552](https://serlo-staging.dev/196552))
* [course-page 196155](https://serlo-staging.dev/196155) ([current revision 196592](https://serlo-staging.dev/196592))
* [course-page 196146](https://serlo-staging.dev/196146) ([current revision 196593](https://serlo-staging.dev/196593))
* [course-page 196542](https://serlo-staging.dev/196542) ([current revision 196600](https://serlo-staging.dev/196600))
* [course-page 196546](https://serlo-staging.dev/196546) ([current revision 196604](https://serlo-staging.dev/196604))
* [course-page 196567](https://serlo-staging.dev/196567) ([current revision 196607](https://serlo-staging.dev/196607))
* [course-page 196057](https://serlo-staging.dev/196057) ([current revision 196611](https://serlo-staging.dev/196611))
* [course-page 196543](https://serlo-staging.dev/196543) ([current revision 196656](https://serlo-staging.dev/196656))
* [course-page 196566](https://serlo-staging.dev/196566) ([current revision 196771](https://serlo-staging.dev/196771))
* [article 56795](https://serlo-staging.dev/56795) ([current revision 196827](https://serlo-staging.dev/196827))
* [course-page 52448](https://serlo-staging.dev/52448) ([current revision 196828](https://serlo-staging.dev/196828))
* [article 152028](https://serlo-staging.dev/152028) ([current revision 196948](https://serlo-staging.dev/196948))
* [course-page 195321](https://serlo-staging.dev/195321) ([current revision 196957](https://serlo-staging.dev/196957))
* [article 170218](https://serlo-staging.dev/170218) ([current revision 196962](https://serlo-staging.dev/196962))
* [article 1775](https://serlo-staging.dev/1775) ([current revision 197265](https://serlo-staging.dev/197265))
* [article 42091](https://serlo-staging.dev/42091) ([current revision 197307](https://serlo-staging.dev/197307))
* [article 43283](https://serlo-staging.dev/43283) ([current revision 197308](https://serlo-staging.dev/197308))
* [article 158205](https://serlo-staging.dev/158205) ([current revision 197316](https://serlo-staging.dev/197316))
* [article 53073](https://serlo-staging.dev/53073) ([current revision 197340](https://serlo-staging.dev/197340))
* [article 90410](https://serlo-staging.dev/90410) ([current revision 197881](https://serlo-staging.dev/197881))
* [text-solution 195378](https://serlo-staging.dev/195378) ([current revision 198061](https://serlo-staging.dev/198061))
* [article 196641](https://serlo-staging.dev/196641) ([current revision 198856](https://serlo-staging.dev/198856))
* [article 160297](https://serlo-staging.dev/160297) ([current revision 198881](https://serlo-staging.dev/198881))
* [article 1969](https://serlo-staging.dev/1969) ([current revision 198970](https://serlo-staging.dev/198970))
* [article 199872](https://serlo-staging.dev/199872) ([current revision 199879](https://serlo-staging.dev/199879))
* [course-page 199316](https://serlo-staging.dev/199316) ([current revision 200106](https://serlo-staging.dev/200106))
* [text-solution 60185](https://serlo-staging.dev/60185) ([current revision 201658](https://serlo-staging.dev/201658))
* [text-exercise 60195](https://serlo-staging.dev/60195) ([current revision 201674](https://serlo-staging.dev/201674))
* [text-solution 72972](https://serlo-staging.dev/72972) ([current revision 201708](https://serlo-staging.dev/201708))
* [article 186051](https://serlo-staging.dev/186051) ([current revision 202081](https://serlo-staging.dev/202081))
* [article 178250](https://serlo-staging.dev/178250) ([current revision 202862](https://serlo-staging.dev/202862))
* [course-page 72260](https://serlo-staging.dev/72260) ([current revision 202896](https://serlo-staging.dev/202896))
* [article 185660](https://serlo-staging.dev/185660) ([current revision 202910](https://serlo-staging.dev/202910))
* [article 203329](https://serlo-staging.dev/203329) ([current revision 203337](https://serlo-staging.dev/203337))
* [article 203664](https://serlo-staging.dev/203664) ([current revision 203669](https://serlo-staging.dev/203669))
* [article 150885](https://serlo-staging.dev/150885) ([current revision 204163](https://serlo-staging.dev/204163))
* [course-page 199305](https://serlo-staging.dev/199305) ([current revision 204288](https://serlo-staging.dev/204288))
* [article 199695](https://serlo-staging.dev/199695) ([current revision 205340](https://serlo-staging.dev/205340))
* [course-page 206245](https://serlo-staging.dev/206245) ([current revision 206253](https://serlo-staging.dev/206253))
* [course-page 206248](https://serlo-staging.dev/206248) ([current revision 206256](https://serlo-staging.dev/206256))
* [course-page 206368](https://serlo-staging.dev/206368) ([current revision 206376](https://serlo-staging.dev/206376))
* [course-page 206369](https://serlo-staging.dev/206369) ([current revision 206377](https://serlo-staging.dev/206377))
* [course-page 203725](https://serlo-staging.dev/203725) ([current revision 207981](https://serlo-staging.dev/207981))
* [course-page 203731](https://serlo-staging.dev/203731) ([current revision 207994](https://serlo-staging.dev/207994))
* [course-page 203727](https://serlo-staging.dev/203727) ([current revision 207996](https://serlo-staging.dev/207996))
* [course-page 161358](https://serlo-staging.dev/161358) ([current revision 208316](https://serlo-staging.dev/208316))
* [course-page 161364](https://serlo-staging.dev/161364) ([current revision 208318](https://serlo-staging.dev/208318))
* [article 2005](https://serlo-staging.dev/2005) ([current revision 208323](https://serlo-staging.dev/208323))
* [article 204851](https://serlo-staging.dev/204851) ([current revision 208694](https://serlo-staging.dev/208694))
* [article 150648](https://serlo-staging.dev/150648) ([current revision 209138](https://serlo-staging.dev/209138))
* [course-page 210270](https://serlo-staging.dev/210270) ([current revision 210280](https://serlo-staging.dev/210280))
* [course-page 210284](https://serlo-staging.dev/210284) ([current revision 210285](https://serlo-staging.dev/210285))
* [article 1679](https://serlo-staging.dev/1679) ([current revision 210897](https://serlo-staging.dev/210897))
* [article 211023](https://serlo-staging.dev/211023) ([current revision 211039](https://serlo-staging.dev/211039))
* [article 181051](https://serlo-staging.dev/181051) ([current revision 212149](https://serlo-staging.dev/212149))
* [article 149480](https://serlo-staging.dev/149480) ([current revision 212417](https://serlo-staging.dev/212417))
* [article 212604](https://serlo-staging.dev/212604) ([current revision 212636](https://serlo-staging.dev/212636))
* [article 212976](https://serlo-staging.dev/212976) ([current revision 213280](https://serlo-staging.dev/213280))
* [article 210777](https://serlo-staging.dev/210777) ([current revision 213803](https://serlo-staging.dev/213803))
* [article 214104](https://serlo-staging.dev/214104) ([current revision 214125](https://serlo-staging.dev/214125))
* [course-page 212169](https://serlo-staging.dev/212169) ([current revision 214566](https://serlo-staging.dev/214566))
* [article 25369](https://serlo-staging.dev/25369) ([current revision 214983](https://serlo-staging.dev/214983))
* [article 215596](https://serlo-staging.dev/215596) ([current revision 215681](https://serlo-staging.dev/215681))
* [article 1769](https://serlo-staging.dev/1769) ([current revision 215689](https://serlo-staging.dev/215689))
* [text-solution 215752](https://serlo-staging.dev/215752) ([current revision 215754](https://serlo-staging.dev/215754))
* [event 147713](https://serlo-staging.dev/147713) ([current revision 215780](https://serlo-staging.dev/215780))
* [course-page 215763](https://serlo-staging.dev/215763) ([current revision 215806](https://serlo-staging.dev/215806))
* [course-page 215805](https://serlo-staging.dev/215805) ([current revision 215848](https://serlo-staging.dev/215848))
* [course-page 215847](https://serlo-staging.dev/215847) ([current revision 215852](https://serlo-staging.dev/215852))
* [course-page 215850](https://serlo-staging.dev/215850) ([current revision 215853](https://serlo-staging.dev/215853))
* [article 215910](https://serlo-staging.dev/215910) ([current revision 215924](https://serlo-staging.dev/215924))
* [article 215941](https://serlo-staging.dev/215941) ([current revision 215943](https://serlo-staging.dev/215943))
* [article 216917](https://serlo-staging.dev/216917) ([current revision 216921](https://serlo-staging.dev/216921))
* [course-page 209964](https://serlo-staging.dev/209964) ([current revision 217517](https://serlo-staging.dev/217517))
* [course-page 217772](https://serlo-staging.dev/217772) ([current revision 217799](https://serlo-staging.dev/217799))
* [course-page 217794](https://serlo-staging.dev/217794) ([current revision 218498](https://serlo-staging.dev/218498))
* [course-page 217795](https://serlo-staging.dev/217795) ([current revision 218518](https://serlo-staging.dev/218518))
* [course-page 217797](https://serlo-staging.dev/217797) ([current revision 219010](https://serlo-staging.dev/219010))
* [course-page 217796](https://serlo-staging.dev/217796) ([current revision 219011](https://serlo-staging.dev/219011))
* [article 209594](https://serlo-staging.dev/209594) ([current revision 219443](https://serlo-staging.dev/219443))
* [article 219101](https://serlo-staging.dev/219101) ([current revision 220157](https://serlo-staging.dev/220157))
* [article 220154](https://serlo-staging.dev/220154) ([current revision 220213](https://serlo-staging.dev/220213))
* [article 205995](https://serlo-staging.dev/205995) ([current revision 220253](https://serlo-staging.dev/220253))
* [article 220146](https://serlo-staging.dev/220146) ([current revision 220571](https://serlo-staging.dev/220571))
* [article 2117](https://serlo-staging.dev/2117) ([current revision 220683](https://serlo-staging.dev/220683))
* [article 86467](https://serlo-staging.dev/86467) ([current revision 220723](https://serlo-staging.dev/220723))
* [article 148759](https://serlo-staging.dev/148759) ([current revision 220724](https://serlo-staging.dev/220724))
* [article 146609](https://serlo-staging.dev/146609) ([current revision 220725](https://serlo-staging.dev/220725))
* [article 146611](https://serlo-staging.dev/146611) ([current revision 220829](https://serlo-staging.dev/220829))
* [article 199237](https://serlo-staging.dev/199237) ([current revision 220831](https://serlo-staging.dev/220831))
* [article 205303](https://serlo-staging.dev/205303) ([current revision 220836](https://serlo-staging.dev/220836))
* [article 203177](https://serlo-staging.dev/203177) ([current revision 220840](https://serlo-staging.dev/220840))
* [article 146610](https://serlo-staging.dev/146610) ([current revision 220955](https://serlo-staging.dev/220955))
* [article 204811](https://serlo-staging.dev/204811) ([current revision 221529](https://serlo-staging.dev/221529))
* [article 1821](https://serlo-staging.dev/1821) ([current revision 221534](https://serlo-staging.dev/221534))
* [article 179161](https://serlo-staging.dev/179161) ([current revision 221546](https://serlo-staging.dev/221546))
* [course-page 133596](https://serlo-staging.dev/133596) ([current revision 222011](https://serlo-staging.dev/222011))
* [article 202125](https://serlo-staging.dev/202125) ([current revision 222497](https://serlo-staging.dev/222497))
* [article 51490](https://serlo-staging.dev/51490) ([current revision 222668](https://serlo-staging.dev/222668))
* [article 1835](https://serlo-staging.dev/1835) ([current revision 222710](https://serlo-staging.dev/222710))
* [article 222470](https://serlo-staging.dev/222470) ([current revision 222734](https://serlo-staging.dev/222734))
* [article 222735](https://serlo-staging.dev/222735) ([current revision 222743](https://serlo-staging.dev/222743))
* [article 181038](https://serlo-staging.dev/181038) ([current revision 222779](https://serlo-staging.dev/222779))
* [article 90412](https://serlo-staging.dev/90412) ([current revision 223073](https://serlo-staging.dev/223073))
* [course-page 221397](https://serlo-staging.dev/221397) ([current revision 223347](https://serlo-staging.dev/223347))
* [article 223492](https://serlo-staging.dev/223492) ([current revision 223499](https://serlo-staging.dev/223499))
* [article 114026](https://serlo-staging.dev/114026) ([current revision 223647](https://serlo-staging.dev/223647))
* [course-page 221142](https://serlo-staging.dev/221142) ([current revision 224222](https://serlo-staging.dev/224222))
* [course-page 221141](https://serlo-staging.dev/221141) ([current revision 224223](https://serlo-staging.dev/224223))
* [course-page 221145](https://serlo-staging.dev/221145) ([current revision 224224](https://serlo-staging.dev/224224))
* [course-page 221394](https://serlo-staging.dev/221394) ([current revision 224225](https://serlo-staging.dev/224225))
* [article 77998](https://serlo-staging.dev/77998) ([current revision 224237](https://serlo-staging.dev/224237))
* [article 134964](https://serlo-staging.dev/134964) ([current revision 224251](https://serlo-staging.dev/224251))
* [course-page 221400](https://serlo-staging.dev/221400) ([current revision 224261](https://serlo-staging.dev/224261))
* [article 1657](https://serlo-staging.dev/1657) ([current revision 224452](https://serlo-staging.dev/224452))
* [text-solution 223711](https://serlo-staging.dev/223711) ([current revision 224461](https://serlo-staging.dev/224461))
* [article 201038](https://serlo-staging.dev/201038) ([current revision 224532](https://serlo-staging.dev/224532))
* [article 1527](https://serlo-staging.dev/1527) ([current revision 224591](https://serlo-staging.dev/224591))
* [course-page 222210](https://serlo-staging.dev/222210) ([current revision 224603](https://serlo-staging.dev/224603))
* [article 224615](https://serlo-staging.dev/224615) ([current revision 224617](https://serlo-staging.dev/224617))
* [course-page 222216](https://serlo-staging.dev/222216) ([current revision 224620](https://serlo-staging.dev/224620))
* [course-page 222214](https://serlo-staging.dev/222214) ([current revision 224637](https://serlo-staging.dev/224637))
* [course-page 222226](https://serlo-staging.dev/222226) ([current revision 224641](https://serlo-staging.dev/224641))
* [article 225024](https://serlo-staging.dev/225024) ([current revision 225051](https://serlo-staging.dev/225051))
* [article 216333](https://serlo-staging.dev/216333) ([current revision 225082](https://serlo-staging.dev/225082))
* [course-page 222218](https://serlo-staging.dev/222218) ([current revision 225090](https://serlo-staging.dev/225090))
* [article 224015](https://serlo-staging.dev/224015) ([current revision 225368](https://serlo-staging.dev/225368))
* [course-page 225447](https://serlo-staging.dev/225447) ([current revision 225460](https://serlo-staging.dev/225460))
* [article 1587](https://serlo-staging.dev/1587) ([current revision 225962](https://serlo-staging.dev/225962))
* [article 1603](https://serlo-staging.dev/1603) ([current revision 226180](https://serlo-staging.dev/226180))
* [course-page 222212](https://serlo-staging.dev/222212) ([current revision 226196](https://serlo-staging.dev/226196))
* [article 210061](https://serlo-staging.dev/210061) ([current revision 226244](https://serlo-staging.dev/226244))
* [article 213334](https://serlo-staging.dev/213334) ([current revision 226245](https://serlo-staging.dev/226245))
* [article 213337](https://serlo-staging.dev/213337) ([current revision 226246](https://serlo-staging.dev/226246))
* [article 169303](https://serlo-staging.dev/169303) ([current revision 226802](https://serlo-staging.dev/226802))
* [article 167340](https://serlo-staging.dev/167340) ([current revision 226807](https://serlo-staging.dev/226807))
* [article 226897](https://serlo-staging.dev/226897) ([current revision 226914](https://serlo-staging.dev/226914))
* [course-page 134381](https://serlo-staging.dev/134381) ([current revision 227030](https://serlo-staging.dev/227030))
* [course-page 134384](https://serlo-staging.dev/134384) ([current revision 227036](https://serlo-staging.dev/227036))
* [course-page 134568](https://serlo-staging.dev/134568) ([current revision 227037](https://serlo-staging.dev/227037))
* [course-page 134561](https://serlo-staging.dev/134561) ([current revision 227039](https://serlo-staging.dev/227039))
* [course-page 72186](https://serlo-staging.dev/72186) ([current revision 227411](https://serlo-staging.dev/227411))
* [course-page 227433](https://serlo-staging.dev/227433) ([current revision 227442](https://serlo-staging.dev/227442))
* [course-page 227452](https://serlo-staging.dev/227452) ([current revision 227460](https://serlo-staging.dev/227460))
* [course-page 227453](https://serlo-staging.dev/227453) ([current revision 227461](https://serlo-staging.dev/227461))
* [article 143547](https://serlo-staging.dev/143547) ([current revision 227509](https://serlo-staging.dev/227509))
* [article 224867](https://serlo-staging.dev/224867) ([current revision 227634](https://serlo-staging.dev/227634))
* [article 209314](https://serlo-staging.dev/209314) ([current revision 227640](https://serlo-staging.dev/227640))
* [course-page 20326](https://serlo-staging.dev/20326) ([current revision 227835](https://serlo-staging.dev/227835))
* [course-page 21725](https://serlo-staging.dev/21725) ([current revision 227836](https://serlo-staging.dev/227836))
* [course-page 21729](https://serlo-staging.dev/21729) ([current revision 227837](https://serlo-staging.dev/227837))
* [course-page 20329](https://serlo-staging.dev/20329) ([current revision 227839](https://serlo-staging.dev/227839))
* [course-page 22074](https://serlo-staging.dev/22074) ([current revision 227840](https://serlo-staging.dev/227840))
* [article 226235](https://serlo-staging.dev/226235) ([current revision 227899](https://serlo-staging.dev/227899))
* [article 225324](https://serlo-staging.dev/225324) ([current revision 228161](https://serlo-staging.dev/228161))
* [article 226236](https://serlo-staging.dev/226236) ([current revision 228253](https://serlo-staging.dev/228253))
* [article 144838](https://serlo-staging.dev/144838) ([current revision 228465](https://serlo-staging.dev/228465))
* [article 210900](https://serlo-staging.dev/210900) ([current revision 228614](https://serlo-staging.dev/228614))
* [article 134821](https://serlo-staging.dev/134821) ([current revision 228618](https://serlo-staging.dev/228618))
* [article 1649](https://serlo-staging.dev/1649) ([current revision 228662](https://serlo-staging.dev/228662))
* [article 227544](https://serlo-staging.dev/227544) ([current revision 228792](https://serlo-staging.dev/228792))
* [event 142218](https://serlo-staging.dev/142218) ([current revision 228807](https://serlo-staging.dev/228807))
* [article 1647](https://serlo-staging.dev/1647) ([current revision 228830](https://serlo-staging.dev/228830))
* [article 1659](https://serlo-staging.dev/1659) ([current revision 229056](https://serlo-staging.dev/229056))
* [article 225632](https://serlo-staging.dev/225632) ([current revision 229278](https://serlo-staging.dev/229278))
* [article 2067](https://serlo-staging.dev/2067) ([current revision 229326](https://serlo-staging.dev/229326))
* [article 1675](https://serlo-staging.dev/1675) ([current revision 229560](https://serlo-staging.dev/229560))
* [article 229428](https://serlo-staging.dev/229428) ([current revision 229775](https://serlo-staging.dev/229775))
* [article 227518](https://serlo-staging.dev/227518) ([current revision 229919](https://serlo-staging.dev/229919))
* [course-page 145632](https://serlo-staging.dev/145632) ([current revision 230114](https://serlo-staging.dev/230114))
* [article 229982](https://serlo-staging.dev/229982) ([current revision 231011](https://serlo-staging.dev/231011))
* [article 1623](https://serlo-staging.dev/1623) ([current revision 231212](https://serlo-staging.dev/231212))
* [article 231195](https://serlo-staging.dev/231195) ([current revision 231224](https://serlo-staging.dev/231224))
* [text-solution 231324](https://serlo-staging.dev/231324) ([current revision 231326](https://serlo-staging.dev/231326))
* [article 227023](https://serlo-staging.dev/227023) ([current revision 231367](https://serlo-staging.dev/231367))
* [article 200617](https://serlo-staging.dev/200617) ([current revision 231388](https://serlo-staging.dev/231388))
* [article 1701](https://serlo-staging.dev/1701) ([current revision 231468](https://serlo-staging.dev/231468))
* [article 231763](https://serlo-staging.dev/231763) ([current revision 231787](https://serlo-staging.dev/231787))
* [course-page 232013](https://serlo-staging.dev/232013) ([current revision 232018](https://serlo-staging.dev/232018))
* [article 201287](https://serlo-staging.dev/201287) ([current revision 232108](https://serlo-staging.dev/232108))
* [article 168870](https://serlo-staging.dev/168870) ([current revision 232716](https://serlo-staging.dev/232716))
* [course-page 217723](https://serlo-staging.dev/217723) ([current revision 232900](https://serlo-staging.dev/232900))
* [article 213046](https://serlo-staging.dev/213046) ([current revision 233779](https://serlo-staging.dev/233779))
* [article 213105](https://serlo-staging.dev/213105) ([current revision 233781](https://serlo-staging.dev/233781))
* [article 213107](https://serlo-staging.dev/213107) ([current revision 233782](https://serlo-staging.dev/233782))
* [article 228138](https://serlo-staging.dev/228138) ([current revision 234112](https://serlo-staging.dev/234112))
* [article 230333](https://serlo-staging.dev/230333) ([current revision 234162](https://serlo-staging.dev/234162))
* [article 230337](https://serlo-staging.dev/230337) ([current revision 234167](https://serlo-staging.dev/234167))
* [course-page 233718](https://serlo-staging.dev/233718) ([current revision 234282](https://serlo-staging.dev/234282))
* [article 230345](https://serlo-staging.dev/230345) ([current revision 234622](https://serlo-staging.dev/234622))
* [article 234551](https://serlo-staging.dev/234551) ([current revision 234679](https://serlo-staging.dev/234679))
* [article 54352](https://serlo-staging.dev/54352) ([current revision 235196](https://serlo-staging.dev/235196))
* [article 234395](https://serlo-staging.dev/234395) ([current revision 235404](https://serlo-staging.dev/235404))
* [course-page 235390](https://serlo-staging.dev/235390) ([current revision 235525](https://serlo-staging.dev/235525))
* [course-page 235391](https://serlo-staging.dev/235391) ([current revision 235526](https://serlo-staging.dev/235526))
* [course-page 233009](https://serlo-staging.dev/233009) ([current revision 235628](https://serlo-staging.dev/235628))
* [course-page 235633](https://serlo-staging.dev/235633) ([current revision 235649](https://serlo-staging.dev/235649))
* [course-page 235532](https://serlo-staging.dev/235532) ([current revision 235716](https://serlo-staging.dev/235716))
* [article 230452](https://serlo-staging.dev/230452) ([current revision 236772](https://serlo-staging.dev/236772))
* [article 230454](https://serlo-staging.dev/230454) ([current revision 236780](https://serlo-staging.dev/236780))
* [article 1873](https://serlo-staging.dev/1873) ([current revision 236834](https://serlo-staging.dev/236834))
* [text-solution 236838](https://serlo-staging.dev/236838) ([current revision 236853](https://serlo-staging.dev/236853))
* [course-page 236212](https://serlo-staging.dev/236212) ([current revision 236888](https://serlo-staging.dev/236888))
* [course-page 236210](https://serlo-staging.dev/236210) ([current revision 236909](https://serlo-staging.dev/236909))
* [article 140481](https://serlo-staging.dev/140481) ([current revision 236957](https://serlo-staging.dev/236957))
* [article 231373](https://serlo-staging.dev/231373) ([current revision 237017](https://serlo-staging.dev/237017))
* [article 236942](https://serlo-staging.dev/236942) ([current revision 237267](https://serlo-staging.dev/237267))
* [course-page 62402](https://serlo-staging.dev/62402) ([current revision 237373](https://serlo-staging.dev/237373))
* [course-page 237339](https://serlo-staging.dev/237339) ([current revision 237453](https://serlo-staging.dev/237453))
* [course-page 237340](https://serlo-staging.dev/237340) ([current revision 237454](https://serlo-staging.dev/237454))
* [course-page 237469](https://serlo-staging.dev/237469) ([current revision 237473](https://serlo-staging.dev/237473))
* [article 201259](https://serlo-staging.dev/201259) ([current revision 237767](https://serlo-staging.dev/237767))
* [article 201374](https://serlo-staging.dev/201374) ([current revision 237769](https://serlo-staging.dev/237769))
* [course-page 236211](https://serlo-staging.dev/236211) ([current revision 237857](https://serlo-staging.dev/237857))
* [article 237673](https://serlo-staging.dev/237673) ([current revision 237885](https://serlo-staging.dev/237885))
* [article 1719](https://serlo-staging.dev/1719) ([current revision 237889](https://serlo-staging.dev/237889))
* [course-page 237985](https://serlo-staging.dev/237985) ([current revision 237991](https://serlo-staging.dev/237991))
* [article 201470](https://serlo-staging.dev/201470) ([current revision 238016](https://serlo-staging.dev/238016))
* [course-page 237487](https://serlo-staging.dev/237487) ([current revision 238022](https://serlo-staging.dev/238022))
* [article 1739](https://serlo-staging.dev/1739) ([current revision 238023](https://serlo-staging.dev/238023))
* [article 202484](https://serlo-staging.dev/202484) ([current revision 238032](https://serlo-staging.dev/238032))
* [course-page 238092](https://serlo-staging.dev/238092) ([current revision 238132](https://serlo-staging.dev/238132))
* [course-page 238093](https://serlo-staging.dev/238093) ([current revision 238133](https://serlo-staging.dev/238133))
* [course-page 238094](https://serlo-staging.dev/238094) ([current revision 238134](https://serlo-staging.dev/238134))
* [course-page 237816](https://serlo-staging.dev/237816) ([current revision 238517](https://serlo-staging.dev/238517))
* [course-page 237813](https://serlo-staging.dev/237813) ([current revision 238604](https://serlo-staging.dev/238604))
* [article 171893](https://serlo-staging.dev/171893) ([current revision 238616](https://serlo-staging.dev/238616))
* [article 169197](https://serlo-staging.dev/169197) ([current revision 238620](https://serlo-staging.dev/238620))
* [article 196626](https://serlo-staging.dev/196626) ([current revision 238627](https://serlo-staging.dev/238627))
* [article 213826](https://serlo-staging.dev/213826) ([current revision 238629](https://serlo-staging.dev/238629))
* [text-solution 208436](https://serlo-staging.dev/208436) ([current revision 238748](https://serlo-staging.dev/238748))
* [article 202957](https://serlo-staging.dev/202957) ([current revision 238887](https://serlo-staging.dev/238887))
* [article 1743](https://serlo-staging.dev/1743) ([current revision 238888](https://serlo-staging.dev/238888))
* [article 203204](https://serlo-staging.dev/203204) ([current revision 238890](https://serlo-staging.dev/238890))
* [article 203940](https://serlo-staging.dev/203940) ([current revision 238892](https://serlo-staging.dev/238892))
* [course-page 237927](https://serlo-staging.dev/237927) ([current revision 239058](https://serlo-staging.dev/239058))
* [article 209131](https://serlo-staging.dev/209131) ([current revision 239162](https://serlo-staging.dev/239162))
* [article 117722](https://serlo-staging.dev/117722) ([current revision 239294](https://serlo-staging.dev/239294))
* [article 43367](https://serlo-staging.dev/43367) ([current revision 239302](https://serlo-staging.dev/239302))
* [course-page 239588](https://serlo-staging.dev/239588) ([current revision 239595](https://serlo-staging.dev/239595))
* [course-page 239571](https://serlo-staging.dev/239571) ([current revision 239608](https://serlo-staging.dev/239608))
* [course-page 239572](https://serlo-staging.dev/239572) ([current revision 239609](https://serlo-staging.dev/239609))
* [course-page 58150](https://serlo-staging.dev/58150) ([current revision 239640](https://serlo-staging.dev/239640))
* [course-page 58152](https://serlo-staging.dev/58152) ([current revision 239657](https://serlo-staging.dev/239657))
* [course-page 90385](https://serlo-staging.dev/90385) ([current revision 239685](https://serlo-staging.dev/239685))
* [article 136597](https://serlo-staging.dev/136597) ([current revision 239687](https://serlo-staging.dev/239687))
* [course-page 239508](https://serlo-staging.dev/239508) ([current revision 239726](https://serlo-staging.dev/239726))
* [course-page 223934](https://serlo-staging.dev/223934) ([current revision 239731](https://serlo-staging.dev/239731))
* [article 222167](https://serlo-staging.dev/222167) ([current revision 239750](https://serlo-staging.dev/239750))
* [article 227314](https://serlo-staging.dev/227314) ([current revision 239788](https://serlo-staging.dev/239788))
* [course-page 223941](https://serlo-staging.dev/223941) ([current revision 239793](https://serlo-staging.dev/239793))
* [course-page 223938](https://serlo-staging.dev/223938) ([current revision 239820](https://serlo-staging.dev/239820))
* [course-page 223937](https://serlo-staging.dev/223937) ([current revision 239821](https://serlo-staging.dev/239821))
* [article 161890](https://serlo-staging.dev/161890) ([current revision 239838](https://serlo-staging.dev/239838))
* [article 127473](https://serlo-staging.dev/127473) ([current revision 239840](https://serlo-staging.dev/239840))
* [article 127479](https://serlo-staging.dev/127479) ([current revision 239868](https://serlo-staging.dev/239868))
* [course-page 223940](https://serlo-staging.dev/223940) ([current revision 239880](https://serlo-staging.dev/239880))
* [article 220139](https://serlo-staging.dev/220139) ([current revision 239895](https://serlo-staging.dev/239895))
* [course-page 239795](https://serlo-staging.dev/239795) ([current revision 239952](https://serlo-staging.dev/239952))
* [article 43256](https://serlo-staging.dev/43256) ([current revision 240114](https://serlo-staging.dev/240114))
* [article 1927](https://serlo-staging.dev/1927) ([current revision 240190](https://serlo-staging.dev/240190))
* [article 240300](https://serlo-staging.dev/240300) ([current revision 240349](https://serlo-staging.dev/240349))
* [article 1801](https://serlo-staging.dev/1801) ([current revision 240409](https://serlo-staging.dev/240409))
* [article 111179](https://serlo-staging.dev/111179) ([current revision 240418](https://serlo-staging.dev/240418))
* [article 240351](https://serlo-staging.dev/240351) ([current revision 240444](https://serlo-staging.dev/240444))
* [article 224320](https://serlo-staging.dev/224320) ([current revision 240858](https://serlo-staging.dev/240858))
* [article 178860](https://serlo-staging.dev/178860) ([current revision 240865](https://serlo-staging.dev/240865))
* [article 2213](https://serlo-staging.dev/2213) ([current revision 240899](https://serlo-staging.dev/240899))
* [article 234735](https://serlo-staging.dev/234735) ([current revision 240958](https://serlo-staging.dev/240958))
* [article 1963](https://serlo-staging.dev/1963) ([current revision 241026](https://serlo-staging.dev/241026))
* [article 1789](https://serlo-staging.dev/1789) ([current revision 241072](https://serlo-staging.dev/241072))
* [article 1935](https://serlo-staging.dev/1935) ([current revision 241129](https://serlo-staging.dev/241129))
* [article 241186](https://serlo-staging.dev/241186) ([current revision 241191](https://serlo-staging.dev/241191))
* [article 2015](https://serlo-staging.dev/2015) ([current revision 241245](https://serlo-staging.dev/241245))
* [article 140933](https://serlo-staging.dev/140933) ([current revision 241355](https://serlo-staging.dev/241355))
* [article 159135](https://serlo-staging.dev/159135) ([current revision 241356](https://serlo-staging.dev/241356))
* [article 1571](https://serlo-staging.dev/1571) ([current revision 241555](https://serlo-staging.dev/241555))
* [article 1929](https://serlo-staging.dev/1929) ([current revision 241616](https://serlo-staging.dev/241616))
* [text-solution 7799](https://serlo-staging.dev/7799) ([current revision 241878](https://serlo-staging.dev/241878))
* [article 25509](https://serlo-staging.dev/25509) ([current revision 242007](https://serlo-staging.dev/242007))
* [article 242060](https://serlo-staging.dev/242060) ([current revision 242064](https://serlo-staging.dev/242064))
* [article 26163](https://serlo-staging.dev/26163) ([current revision 242082](https://serlo-staging.dev/242082))
* [article 27801](https://serlo-staging.dev/27801) ([current revision 242132](https://serlo-staging.dev/242132))
* [article 239022](https://serlo-staging.dev/239022) ([current revision 242158](https://serlo-staging.dev/242158))
* [article 199231](https://serlo-staging.dev/199231) ([current revision 242222](https://serlo-staging.dev/242222))
* [article 29577](https://serlo-staging.dev/29577) ([current revision 242538](https://serlo-staging.dev/242538))
* [article 32567](https://serlo-staging.dev/32567) ([current revision 242562](https://serlo-staging.dev/242562))
* [article 34019](https://serlo-staging.dev/34019) ([current revision 242608](https://serlo-staging.dev/242608))
* [article 28557](https://serlo-staging.dev/28557) ([current revision 242806](https://serlo-staging.dev/242806))
* [article 36160](https://serlo-staging.dev/36160) ([current revision 242813](https://serlo-staging.dev/242813))
* [article 35135](https://serlo-staging.dev/35135) ([current revision 242867](https://serlo-staging.dev/242867))
* [article 1767](https://serlo-staging.dev/1767) ([current revision 242870](https://serlo-staging.dev/242870))
* [article 241104](https://serlo-staging.dev/241104) ([current revision 242899](https://serlo-staging.dev/242899))
* [article 240050](https://serlo-staging.dev/240050) ([current revision 242910](https://serlo-staging.dev/242910))
* [article 242841](https://serlo-staging.dev/242841) ([current revision 242981](https://serlo-staging.dev/242981))
* [article 186937](https://serlo-staging.dev/186937) ([current revision 243041](https://serlo-staging.dev/243041))
* [article 240630](https://serlo-staging.dev/240630) ([current revision 243057](https://serlo-staging.dev/243057))
* [article 230163](https://serlo-staging.dev/230163) ([current revision 243278](https://serlo-staging.dev/243278))
* [article 84541](https://serlo-staging.dev/84541) ([current revision 243335](https://serlo-staging.dev/243335))
* [article 1565](https://serlo-staging.dev/1565) ([current revision 243496](https://serlo-staging.dev/243496))
* [article 100130](https://serlo-staging.dev/100130) ([current revision 243587](https://serlo-staging.dev/243587))
* [article 233786](https://serlo-staging.dev/233786) ([current revision 243645](https://serlo-staging.dev/243645))
* [article 243577](https://serlo-staging.dev/243577) ([current revision 243977](https://serlo-staging.dev/243977))
* [article 36268](https://serlo-staging.dev/36268) ([current revision 243988](https://serlo-staging.dev/243988))
* [article 1945](https://serlo-staging.dev/1945) ([current revision 244060](https://serlo-staging.dev/244060))
* [article 241559](https://serlo-staging.dev/241559) ([current revision 244132](https://serlo-staging.dev/244132))
* [article 176956](https://serlo-staging.dev/176956) ([current revision 244277](https://serlo-staging.dev/244277))
* [article 27826](https://serlo-staging.dev/27826) ([current revision 244341](https://serlo-staging.dev/244341))
* [article 51384](https://serlo-staging.dev/51384) ([current revision 244423](https://serlo-staging.dev/244423))
* [article 232392](https://serlo-staging.dev/232392) ([current revision 244600](https://serlo-staging.dev/244600))
* [article 231956](https://serlo-staging.dev/231956) ([current revision 244601](https://serlo-staging.dev/244601))
* [article 55437](https://serlo-staging.dev/55437) ([current revision 244650](https://serlo-staging.dev/244650))
* [article 55476](https://serlo-staging.dev/55476) ([current revision 244654](https://serlo-staging.dev/244654))
* [article 243981](https://serlo-staging.dev/243981) ([current revision 244716](https://serlo-staging.dev/244716))
* [article 53475](https://serlo-staging.dev/53475) ([current revision 244796](https://serlo-staging.dev/244796))
* [article 244538](https://serlo-staging.dev/244538) ([current revision 244832](https://serlo-staging.dev/244832))
* [article 244568](https://serlo-staging.dev/244568) ([current revision 244834](https://serlo-staging.dev/244834))
* [article 43048](https://serlo-staging.dev/43048) ([current revision 245352](https://serlo-staging.dev/245352))
* [article 238693](https://serlo-staging.dev/238693) ([current revision 245555](https://serlo-staging.dev/245555))
* [article 245448](https://serlo-staging.dev/245448) ([current revision 245756](https://serlo-staging.dev/245756))
* [article 226222](https://serlo-staging.dev/226222) ([current revision 245912](https://serlo-staging.dev/245912))
* [article 244510](https://serlo-staging.dev/244510) ([current revision 245958](https://serlo-staging.dev/245958))
* [event 196426](https://serlo-staging.dev/196426) ([current revision 246085](https://serlo-staging.dev/246085))
* [article 224619](https://serlo-staging.dev/224619) ([current revision 246684](https://serlo-staging.dev/246684))
* [article 190844](https://serlo-staging.dev/190844) ([current revision 247171](https://serlo-staging.dev/247171))
* [event 146500](https://serlo-staging.dev/146500) ([current revision 247191](https://serlo-staging.dev/247191))
* [article 238459](https://serlo-staging.dev/238459) ([current revision 247279](https://serlo-staging.dev/247279))
* [article 209254](https://serlo-staging.dev/209254) ([current revision 247359](https://serlo-staging.dev/247359))
* [article 224376](https://serlo-staging.dev/224376) ([current revision 247595](https://serlo-staging.dev/247595))
* [text-exercise 8997](https://serlo-staging.dev/8997) ([current revision 247636](https://serlo-staging.dev/247636))
* [article 244678](https://serlo-staging.dev/244678) ([current revision 248070](https://serlo-staging.dev/248070))
* [article 66440](https://serlo-staging.dev/66440) ([current revision 248098](https://serlo-staging.dev/248098))
* [article 1535](https://serlo-staging.dev/1535) ([current revision 248219](https://serlo-staging.dev/248219))
* [article 1595](https://serlo-staging.dev/1595) ([current revision 248552](https://serlo-staging.dev/248552))
* [event 146498](https://serlo-staging.dev/146498) ([current revision 248786](https://serlo-staging.dev/248786))
* [event 146512](https://serlo-staging.dev/146512) ([current revision 248787](https://serlo-staging.dev/248787))
* [event 150501](https://serlo-staging.dev/150501) ([current revision 248788](https://serlo-staging.dev/248788))
* [article 1559](https://serlo-staging.dev/1559) ([current revision 248816](https://serlo-staging.dev/248816))
* [article 1745](https://serlo-staging.dev/1745) ([current revision 248828](https://serlo-staging.dev/248828))
* [article 2033](https://serlo-staging.dev/2033) ([current revision 248853](https://serlo-staging.dev/248853))
* [article 46787](https://serlo-staging.dev/46787) ([current revision 248889](https://serlo-staging.dev/248889))
* [article 46629](https://serlo-staging.dev/46629) ([current revision 248891](https://serlo-staging.dev/248891))
* [article 36070](https://serlo-staging.dev/36070) ([current revision 248897](https://serlo-staging.dev/248897))
* [article 1553](https://serlo-staging.dev/1553) ([current revision 248981](https://serlo-staging.dev/248981))
* [article 214562](https://serlo-staging.dev/214562) ([current revision 248995](https://serlo-staging.dev/248995))
* [article 1577](https://serlo-staging.dev/1577) ([current revision 248996](https://serlo-staging.dev/248996))

# Entites with plugin blockquote

* [course-page 145842](https://serlo-staging.dev/145842) ([current revision 146879](https://serlo-staging.dev/146879))
* [text-exercise 149487](https://serlo-staging.dev/149487) ([current revision 150043](https://serlo-staging.dev/150043))
* [article 153253](https://serlo-staging.dev/153253) ([current revision 153259](https://serlo-staging.dev/153259))
* [article 153396](https://serlo-staging.dev/153396) ([current revision 153423](https://serlo-staging.dev/153423))
* [article 153437](https://serlo-staging.dev/153437) ([current revision 153438](https://serlo-staging.dev/153438))
* [article 153472](https://serlo-staging.dev/153472) ([current revision 153488](https://serlo-staging.dev/153488))
* [article 153915](https://serlo-staging.dev/153915) ([current revision 153927](https://serlo-staging.dev/153927))
* [article 158346](https://serlo-staging.dev/158346) ([current revision 159271](https://serlo-staging.dev/159271))
* [text-exercise 85711](https://serlo-staging.dev/85711) ([current revision 159826](https://serlo-staging.dev/159826))
* [course-page 160560](https://serlo-staging.dev/160560) ([current revision 160571](https://serlo-staging.dev/160571))
* [course-page 160581](https://serlo-staging.dev/160581) ([current revision 160591](https://serlo-staging.dev/160591))
* [text-exercise 161294](https://serlo-staging.dev/161294) ([current revision 164220](https://serlo-staging.dev/164220))
* [text-solution 164622](https://serlo-staging.dev/164622) ([current revision 166886](https://serlo-staging.dev/166886))
* [article 167847](https://serlo-staging.dev/167847) ([current revision 167922](https://serlo-staging.dev/167922))
* [text-exercise 168267](https://serlo-staging.dev/168267) ([current revision 169648](https://serlo-staging.dev/169648))
* [text-exercise 169709](https://serlo-staging.dev/169709) ([current revision 169711](https://serlo-staging.dev/169711))
* [text-solution 169716](https://serlo-staging.dev/169716) ([current revision 169718](https://serlo-staging.dev/169718))
* [course-page 169720](https://serlo-staging.dev/169720) ([current revision 169733](https://serlo-staging.dev/169733))
* [text-exercise 169713](https://serlo-staging.dev/169713) ([current revision 169809](https://serlo-staging.dev/169809))
* [text-solution 47202](https://serlo-staging.dev/47202) ([current revision 170464](https://serlo-staging.dev/170464))
* [article 167755](https://serlo-staging.dev/167755) ([current revision 170467](https://serlo-staging.dev/170467))
* [article 168400](https://serlo-staging.dev/168400) ([current revision 170632](https://serlo-staging.dev/170632))
* [article 170872](https://serlo-staging.dev/170872) ([current revision 171950](https://serlo-staging.dev/171950))
* [article 2105](https://serlo-staging.dev/2105) ([current revision 172953](https://serlo-staging.dev/172953))
* [article 168163](https://serlo-staging.dev/168163) ([current revision 173486](https://serlo-staging.dev/173486))
* [course-page 31241](https://serlo-staging.dev/31241) ([current revision 174401](https://serlo-staging.dev/174401))
* [course-page 177831](https://serlo-staging.dev/177831) ([current revision 177840](https://serlo-staging.dev/177840))
* [text-solution 106241](https://serlo-staging.dev/106241) ([current revision 179371](https://serlo-staging.dev/179371))
* [article 167497](https://serlo-staging.dev/167497) ([current revision 179907](https://serlo-staging.dev/179907))
* [text-exercise 182017](https://serlo-staging.dev/182017) ([current revision 183303](https://serlo-staging.dev/183303))
* [text-solution 183392](https://serlo-staging.dev/183392) ([current revision 184270](https://serlo-staging.dev/184270))
* [article 186621](https://serlo-staging.dev/186621) ([current revision 186626](https://serlo-staging.dev/186626))
* [course-page 188326](https://serlo-staging.dev/188326) ([current revision 188385](https://serlo-staging.dev/188385))
* [article 188985](https://serlo-staging.dev/188985) ([current revision 191254](https://serlo-staging.dev/191254))
* [article 174232](https://serlo-staging.dev/174232) ([current revision 193814](https://serlo-staging.dev/193814))
* [course-page 193828](https://serlo-staging.dev/193828) ([current revision 193829](https://serlo-staging.dev/193829))
* [course-page 193817](https://serlo-staging.dev/193817) ([current revision 193842](https://serlo-staging.dev/193842))
* [course-page 178835](https://serlo-staging.dev/178835) ([current revision 195226](https://serlo-staging.dev/195226))
* [article 195560](https://serlo-staging.dev/195560) ([current revision 195823](https://serlo-staging.dev/195823))
* [course-page 196057](https://serlo-staging.dev/196057) ([current revision 196611](https://serlo-staging.dev/196611))
* [course-page 195589](https://serlo-staging.dev/195589) ([current revision 196613](https://serlo-staging.dev/196613))
* [article 196646](https://serlo-staging.dev/196646) ([current revision 196666](https://serlo-staging.dev/196666))
* [article 167816](https://serlo-staging.dev/167816) ([current revision 197032](https://serlo-staging.dev/197032))
* [article 181857](https://serlo-staging.dev/181857) ([current revision 198744](https://serlo-staging.dev/198744))
* [article 201230](https://serlo-staging.dev/201230) ([current revision 201233](https://serlo-staging.dev/201233))
* [course-page 201507](https://serlo-staging.dev/201507) ([current revision 201586](https://serlo-staging.dev/201586))
* [article 185670](https://serlo-staging.dev/185670) ([current revision 202969](https://serlo-staging.dev/202969))
* [article 200819](https://serlo-staging.dev/200819) ([current revision 202997](https://serlo-staging.dev/202997))
* [article 203263](https://serlo-staging.dev/203263) ([current revision 203265](https://serlo-staging.dev/203265))
* [article 184341](https://serlo-staging.dev/184341) ([current revision 204156](https://serlo-staging.dev/204156))
* [article 183223](https://serlo-staging.dev/183223) ([current revision 204647](https://serlo-staging.dev/204647))
* [article 204982](https://serlo-staging.dev/204982) ([current revision 204983](https://serlo-staging.dev/204983))
* [course-page 205022](https://serlo-staging.dev/205022) ([current revision 205026](https://serlo-staging.dev/205026))
* [course-page 205023](https://serlo-staging.dev/205023) ([current revision 205031](https://serlo-staging.dev/205031))
* [course-page 179604](https://serlo-staging.dev/179604) ([current revision 205308](https://serlo-staging.dev/205308))
* [article 188454](https://serlo-staging.dev/188454) ([current revision 209878](https://serlo-staging.dev/209878))
* [article 203882](https://serlo-staging.dev/203882) ([current revision 209883](https://serlo-staging.dev/209883))
* [article 204063](https://serlo-staging.dev/204063) ([current revision 209884](https://serlo-staging.dev/209884))
* [article 210007](https://serlo-staging.dev/210007) ([current revision 210013](https://serlo-staging.dev/210013))
* [course-page 210638](https://serlo-staging.dev/210638) ([current revision 210647](https://serlo-staging.dev/210647))
* [course-page 210640](https://serlo-staging.dev/210640) ([current revision 210649](https://serlo-staging.dev/210649))
* [course-page 210641](https://serlo-staging.dev/210641) ([current revision 210650](https://serlo-staging.dev/210650))
* [course-page 210655](https://serlo-staging.dev/210655) ([current revision 210663](https://serlo-staging.dev/210663))
* [course-page 210657](https://serlo-staging.dev/210657) ([current revision 210665](https://serlo-staging.dev/210665))
* [course-page 210658](https://serlo-staging.dev/210658) ([current revision 210666](https://serlo-staging.dev/210666))
* [course-page 210780](https://serlo-staging.dev/210780) ([current revision 210790](https://serlo-staging.dev/210790))
* [course-page 210783](https://serlo-staging.dev/210783) ([current revision 210793](https://serlo-staging.dev/210793))
* [course-page 210787](https://serlo-staging.dev/210787) ([current revision 210797](https://serlo-staging.dev/210797))
* [course-page 210798](https://serlo-staging.dev/210798) ([current revision 210806](https://serlo-staging.dev/210806))
* [course-page 210801](https://serlo-staging.dev/210801) ([current revision 210809](https://serlo-staging.dev/210809))
* [course-page 210805](https://serlo-staging.dev/210805) ([current revision 210813](https://serlo-staging.dev/210813))
* [course-page 196058](https://serlo-staging.dev/196058) ([current revision 212805](https://serlo-staging.dev/212805))
* [article 212396](https://serlo-staging.dev/212396) ([current revision 213121](https://serlo-staging.dev/213121))
* [article 212976](https://serlo-staging.dev/212976) ([current revision 213280](https://serlo-staging.dev/213280))
* [article 201375](https://serlo-staging.dev/201375) ([current revision 213797](https://serlo-staging.dev/213797))
* [article 205298](https://serlo-staging.dev/205298) ([current revision 213799](https://serlo-staging.dev/213799))
* [article 210777](https://serlo-staging.dev/210777) ([current revision 213803](https://serlo-staging.dev/213803))
* [article 201974](https://serlo-staging.dev/201974) ([current revision 213833](https://serlo-staging.dev/213833))
* [article 215903](https://serlo-staging.dev/215903) ([current revision 215938](https://serlo-staging.dev/215938))
* [article 215941](https://serlo-staging.dev/215941) ([current revision 215943](https://serlo-staging.dev/215943))
* [article 214242](https://serlo-staging.dev/214242) ([current revision 216186](https://serlo-staging.dev/216186))
* [article 216917](https://serlo-staging.dev/216917) ([current revision 216921](https://serlo-staging.dev/216921))
* [text-solution 217263](https://serlo-staging.dev/217263) ([current revision 217268](https://serlo-staging.dev/217268))
* [article 158556](https://serlo-staging.dev/158556) ([current revision 219189](https://serlo-staging.dev/219189))
* [article 199237](https://serlo-staging.dev/199237) ([current revision 220831](https://serlo-staging.dev/220831))
* [article 205303](https://serlo-staging.dev/205303) ([current revision 220836](https://serlo-staging.dev/220836))
* [article 203177](https://serlo-staging.dev/203177) ([current revision 220840](https://serlo-staging.dev/220840))
* [article 220880](https://serlo-staging.dev/220880) ([current revision 220958](https://serlo-staging.dev/220958))
* [course-page 136399](https://serlo-staging.dev/136399) ([current revision 222176](https://serlo-staging.dev/222176))
* [course-page 132723](https://serlo-staging.dev/132723) ([current revision 222178](https://serlo-staging.dev/222178))
* [article 222611](https://serlo-staging.dev/222611) ([current revision 222626](https://serlo-staging.dev/222626))
* [article 1693](https://serlo-staging.dev/1693) ([current revision 223403](https://serlo-staging.dev/223403))
* [course-page 120589](https://serlo-staging.dev/120589) ([current revision 223987](https://serlo-staging.dev/223987))
* [course-page 145638](https://serlo-staging.dev/145638) ([current revision 224141](https://serlo-staging.dev/224141))
* [article 223914](https://serlo-staging.dev/223914) ([current revision 224306](https://serlo-staging.dev/224306))
* [article 201038](https://serlo-staging.dev/201038) ([current revision 224532](https://serlo-staging.dev/224532))
* [course-page 222214](https://serlo-staging.dev/222214) ([current revision 224637](https://serlo-staging.dev/224637))
* [text-solution 224832](https://serlo-staging.dev/224832) ([current revision 224834](https://serlo-staging.dev/224834))
* [article 226501](https://serlo-staging.dev/226501) ([current revision 226553](https://serlo-staging.dev/226553))
* [article 193313](https://serlo-staging.dev/193313) ([current revision 226810](https://serlo-staging.dev/226810))
* [article 134867](https://serlo-staging.dev/134867) ([current revision 226927](https://serlo-staging.dev/226927))
* [text-solution 228168](https://serlo-staging.dev/228168) ([current revision 228442](https://serlo-staging.dev/228442))
* [article 210900](https://serlo-staging.dev/210900) ([current revision 228614](https://serlo-staging.dev/228614))
* [course-page 229603](https://serlo-staging.dev/229603) ([current revision 229947](https://serlo-staging.dev/229947))
* [course-page 229605](https://serlo-staging.dev/229605) ([current revision 229988](https://serlo-staging.dev/229988))
* [course-page 229609](https://serlo-staging.dev/229609) ([current revision 229999](https://serlo-staging.dev/229999))
* [course-page 119277](https://serlo-staging.dev/119277) ([current revision 230094](https://serlo-staging.dev/230094))
* [article 213046](https://serlo-staging.dev/213046) ([current revision 233779](https://serlo-staging.dev/233779))
* [article 211859](https://serlo-staging.dev/211859) ([current revision 233780](https://serlo-staging.dev/233780))
* [article 213105](https://serlo-staging.dev/213105) ([current revision 233781](https://serlo-staging.dev/233781))
* [course-page 222284](https://serlo-staging.dev/222284) ([current revision 234328](https://serlo-staging.dev/234328))
* [course-page 234472](https://serlo-staging.dev/234472) ([current revision 234480](https://serlo-staging.dev/234480))
* [course-page 235564](https://serlo-staging.dev/235564) ([current revision 236289](https://serlo-staging.dev/236289))
* [text-solution 227048](https://serlo-staging.dev/227048) ([current revision 237444](https://serlo-staging.dev/237444))
* [course-page 237924](https://serlo-staging.dev/237924) ([current revision 237930](https://serlo-staging.dev/237930))
* [course-page 237926](https://serlo-staging.dev/237926) ([current revision 237932](https://serlo-staging.dev/237932))
* [course-page 237982](https://serlo-staging.dev/237982) ([current revision 237988](https://serlo-staging.dev/237988))
* [course-page 237983](https://serlo-staging.dev/237983) ([current revision 237989](https://serlo-staging.dev/237989))
* [course-page 237984](https://serlo-staging.dev/237984) ([current revision 237990](https://serlo-staging.dev/237990))
* [article 202005](https://serlo-staging.dev/202005) ([current revision 238018](https://serlo-staging.dev/238018))
* [course-page 237816](https://serlo-staging.dev/237816) ([current revision 238517](https://serlo-staging.dev/238517))
* [article 231621](https://serlo-staging.dev/231621) ([current revision 239036](https://serlo-staging.dev/239036))
* [course-page 237925](https://serlo-staging.dev/237925) ([current revision 239057](https://serlo-staging.dev/239057))
* [course-page 237927](https://serlo-staging.dev/237927) ([current revision 239058](https://serlo-staging.dev/239058))
* [article 227314](https://serlo-staging.dev/227314) ([current revision 239788](https://serlo-staging.dev/239788))
* [article 240351](https://serlo-staging.dev/240351) ([current revision 240444](https://serlo-staging.dev/240444))
* [article 178860](https://serlo-staging.dev/178860) ([current revision 240865](https://serlo-staging.dev/240865))
* [article 200685](https://serlo-staging.dev/200685) ([current revision 242224](https://serlo-staging.dev/242224))
* [article 27475](https://serlo-staging.dev/27475) ([current revision 242408](https://serlo-staging.dev/242408))
* [article 27403](https://serlo-staging.dev/27403) ([current revision 242418](https://serlo-staging.dev/242418))
* [article 214865](https://serlo-staging.dev/214865) ([current revision 242422](https://serlo-staging.dev/242422))
* [article 27566](https://serlo-staging.dev/27566) ([current revision 242516](https://serlo-staging.dev/242516))
* [article 33921](https://serlo-staging.dev/33921) ([current revision 242609](https://serlo-staging.dev/242609))
* [article 239733](https://serlo-staging.dev/239733) ([current revision 242869](https://serlo-staging.dev/242869))
* [article 202973](https://serlo-staging.dev/202973) ([current revision 242966](https://serlo-staging.dev/242966))
* [article 243790](https://serlo-staging.dev/243790) ([current revision 243844](https://serlo-staging.dev/243844))
* [article 176956](https://serlo-staging.dev/176956) ([current revision 244277](https://serlo-staging.dev/244277))
* [course-page 244309](https://serlo-staging.dev/244309) ([current revision 244474](https://serlo-staging.dev/244474))
* [article 243867](https://serlo-staging.dev/243867) ([current revision 244502](https://serlo-staging.dev/244502))
* [article 53415](https://serlo-staging.dev/53415) ([current revision 244576](https://serlo-staging.dev/244576))
* [text-solution 195446](https://serlo-staging.dev/195446) ([current revision 244870](https://serlo-staging.dev/244870))
* [text-solution 192951](https://serlo-staging.dev/192951) ([current revision 245225](https://serlo-staging.dev/245225))
* [article 241639](https://serlo-staging.dev/241639) ([current revision 245332](https://serlo-staging.dev/245332))
* [course-page 245715](https://serlo-staging.dev/245715) ([current revision 245722](https://serlo-staging.dev/245722))
* [article 226222](https://serlo-staging.dev/226222) ([current revision 245912](https://serlo-staging.dev/245912))
* [article 245690](https://serlo-staging.dev/245690) ([current revision 246004](https://serlo-staging.dev/246004))
* [article 210119](https://serlo-staging.dev/210119) ([current revision 247266](https://serlo-staging.dev/247266))
* [article 209254](https://serlo-staging.dev/209254) ([current revision 247359](https://serlo-staging.dev/247359))
* [article 214562](https://serlo-staging.dev/214562) ([current revision 248995](https://serlo-staging.dev/248995))